## Extracting Words from Predicted Characters 

The character extraction pipeline is currently unable to recognize spaces and as a result is unable to differentiate between distinct words. This is a starting point to determine how such an approach would work.

Before delving into this, a quick reminder that our pipeline at the moment is : 'Cover Image' -- 'A list of continuous extracted characters' -- 'A continuous string of predcicted characters'.

This continous string of all predicted characters is what we feeding into this model with the expected output being a list of predicted words that make up this string.

Two things to note:

##### 1. We need a more substantial data to better train this model. As of now we are only using data of around 4000 most common words in Italian language. Books in Italian would help the model learn more about frequencies of different words and assign proper weights accordingly

##### 2. The model is a little simplistic where we are training from scratch, mostly this was due to us wanting to learn how exactly a word segementation model works. A different approach would be to use pre calculated n-gram models and use those for predictions

In [1]:
import re
import math
import string
from collections import Counter
from __future__ import division
import random

## Tokenize the data (Input File)

We take the input file we using to train our model (for instance a book), tokenize it and store the statistics of word frequency in a counter.

In [2]:
# tokenizes the data
def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower()) 

In [3]:
text = open('./nlp/template.txt').read()
words = tokens(text)
len(words)

7126

In [4]:
# printing some of the tokens, them being at the top has nothing to do 
# with freqency -- they just show up at the beginning of the document.
print(words[:10])

['alcune', 'lettere', 'mercantili', 'toscane', 'da', 'colonie', 'genovesi', 'alla', 'fine', 'del']


## Convert the text to a bag of words

In [5]:
# printing the most common words, this shows the words with highest
# frequency throughout the document
Counter(words).most_common(10)

[('di', 369),
 ('la', 150),
 ('de', 140),
 ('il', 132),
 ('della', 132),
 ('e', 132),
 ('poesie', 122),
 ('del', 108),
 ('relazione', 108),
 ('rime', 95)]

In [6]:
counts = Counter(words)

## Spelling Correction

In [7]:
def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

In [8]:
# define valid characters, at present we considering only letters
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [9]:
def known(words):
    "Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in counts}

In [10]:
def correct(word):
    "Find the best spelling correction for this word."
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=counts.get)

The edit functions as explained in the comments return words that perfectly match the input word -- edits0() or are at a distance of 1 or 2 edits -- edits1() and edits2()

In [11]:
def edits0(word): 
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}

def edits2(word):
    "Return all strings that are two edits away from this word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def edits1(word):
    "Return all strings that are one edit away from this word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [12]:
def correct_text(text):
    "Correct all the words within a text, returning the corrected text."
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    "Spell-correct word in match, and preserve proper upper/lower/title case."
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    "Return the case-function appropriate for text: upper, lower, title, or just str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [13]:
# The italian translation for "This is an example" is "Cio questo è un esempio"
# We modify a few letter and see how the correct_text() function works

correct_text('Ciago quesyto è uyn gesempio')

'Cio questo è un esempio'

For the simple example above, our model was able to auto correct the sentence accurately.

## Calculate Probabilities of Word Sequences

The first step would be to calculate probabilities of individual words.

In [14]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(counter.values())
    return lambda x: counter[x]/N

P = pdist(counts)

In [15]:
for w in tokens('Cio questo è un esempio'):
    print(P(w), w)

0.0004209935447656469 cio
0.000140331181588549 questo
0.003648610721302273 un
0.000140331181588549 esempio


Now we can use the above result to calculate the probabilities of word sequences but it would give incorrect results because we would be assuming that each word is being drawn independently of other words.

We use the product of individual probabilities to get a better estimate of the probability of a word sequence. 

In [16]:
def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return product(P(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

In [17]:
tests = ['cio',
         'cio questo',
         'cio questo un',
         'cio questo un esempio', 
         'cio olaquesto unesempio ']

for test in tests:
    print(Pwords(tokens(test)), test)

0.0004209935447656469 cio
5.907852157811493e-08 cio questo
2.155545272285978e-10 cio questo un
3.0249021502750184e-14 cio questo un esempio
0.0 cio olaquesto unesempio 


We see that the last entry which was a made up sequence of words gives a 0 probability, while the others have low but non-zero ones.

## Word Segmentation

This is the section that is most useful for us. The thinking is that since we cannot detect spaces in the character extraction process, can we take an input of continuous letters with no spaces and recover the sequence of words that make up the input.

For e.g. If the page has "Ugo Foscolo" and we can only get 'ugofoscolo' at the end of our pipeline, can we write a function split such that 

split('ugofoscolo')= 'ugo','foscolo'?

In [18]:
# using memoization to cut down on time by caching results
def memo(f):
    "Memoize function f, whose args must all be hashable."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [19]:
max_word_len = max(len(w) for w in counts)
max_word_len

16

The longest word we seen so far is 16, so we put a limit on the possible length for the first word in a split to a few more than the max_word_len obtained.

For example if we splitting 'verylongword' into (split1, split2) -- we put a limit on how long split1 can be.

In [20]:
def splits(text, start=0, L=max_word_len+4):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

An example of the outputs obtained from the split function can be seen as follows:

In [21]:
print(splits('laspagna'))
print(splits('drammaticaitaliana', 1, 4))

[('', 'laspagna'), ('l', 'aspagna'), ('la', 'spagna'), ('las', 'pagna'), ('lasp', 'agna'), ('laspa', 'gna'), ('laspag', 'na'), ('laspagn', 'a'), ('laspagna', '')]
[('d', 'rammaticaitaliana'), ('dr', 'ammaticaitaliana'), ('dra', 'mmaticaitaliana'), ('dram', 'maticaitaliana')]


In [22]:
@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        
        # we use our previously defined Pwords function to get the most probable
        # split amongst all candidate splits
        return max(candidates, key=Pwords)

In [23]:
# segment la spagna - Spain
segment('laspagna')

['la', 'spagna']

In [24]:
# trying on author Ugo Foscolo
segment('ugofoscolo')

['ugo', 'foscolo']

In [25]:
# Trying on 'Della nuova scuola drammatica italiana'
segment('Dellanuovascuoladrammaticaitaliana')

['D', 'ella', 'nuova', 'scuola', 'drammatica', 'italiana']

In [26]:
# trying on del bailaggio di Costantinopoli
segment('delbailaggiodiCostantinopoli')

['d',
 'e',
 'l',
 'b',
 'a',
 'i',
 'l',
 'a',
 'g',
 'g',
 'i',
 'o',
 'd',
 'i',
 'C',
 'o',
 'sta',
 'n',
 'ti',
 'no',
 'po',
 'li']

Now, trying the same string as above without the word "Constantinopoli":

In [27]:
# trying on del bailaggio di (Same as above without Constantinopoli)
segment('delbailaggiodi')

['del', 'bailaggio', 'di']

A simple explanation of why we get much better segementation without one word can be seen in the freqency of the constituent words as follows:

In [28]:
for s in ['del', 'bailaggio', 'di', 'Constantinopoli']:
    print('"{}" -- {} occurences'.format(s,counts[s]))

"del" -- 108 occurences
"bailaggio" -- 1 occurences
"di" -- 369 occurences
"Constantinopoli" -- 0 occurences


## Conclusions

As we see from the results, we get satisfactory output when dealing with small and simpler phrases. But when trying to segment bigger strings with less commonly occuring words the results are not very useful.

This can be offset to a certain extent by training our model on a much larger corpus of Italian Texts. But even then results would be skewed by a very large string which is what our pipeline offers at present.

A simple way to improve the results would be to **use the cluster labels that we generated in the character extraction notebook ('3_Character_Extraction') where we clustered the points based on their y coordinates.** 

Since this effectively translates into characters demarcated into lines of text, we decided to preserve the label for each character through the pipeline and then use those labels to create groups of predicted characters. For instance an image with three lines of text, after prediction would give three different strings (of predicted characters).

Each of these strings would be passed through the segmentation function above and the resulting lists would be merged to produce a list of words (in order) that are the final predictions.

## Reference : Peter Norvig's [notebook](https://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb) on statistical NLP.